In [1]:
from pathlib import Path

from tensile_test import TensileTest, HardeningLawFitter
from tensile_test.utils import read_non_uniform_csv
from damask_parse import read_table
from plotly import graph_objects as go

In [2]:
def read_tensile_test_data(path):
    'Read tensile test data CSV into a list of dict.'

    path = Path(path)
    delimiter = ','

    # Read data:
    headers, arr = read_non_uniform_csv(
        path, delimiter=delimiter, skip_rows=2, header_row=1)

    # Get orientations for each stress/strain column pair:
    with path.open() as handle:
        ln = handle.readline()
        oris = [int(i.split()[0]) for i in ln.split(delimiter)[::2]]

    map_keys = {
        'eng. strain': 'eng_strain',
        'eng. stress /Mpa': 'eng_stress',
    }
    out = [
        {
            'orientation': ori,
            map_keys[headers[ori_idx]]: arr[:, ori_idx],
            map_keys[headers[ori_idx + 1]]: arr[:, ori_idx + 1],
        }
        for ori_idx, ori in enumerate(oris)
    ]

    return out

In [3]:
# Load the experimental tensile test data
data_path = Path('data/experimental/surfalex_tensile_test_data.csv')
data = read_tensile_test_data(data_path)

In [4]:
# Get the stress strain data for the first orientation (zero degrees)
ori_idx = 0 

# Generate a fitting object to help with the fitting process and visualise the results:
fitter = HardeningLawFitter(
    exp_stress=data[ori_idx]['eng_stress'],
    exp_strain=data[ori_idx]['eng_strain'],
    trial_taylor_factor=2.5
)

# Visualise the fitting object:
fitter.show()

    'data': [{'line': {'color': 'rgb(31, 119, 180)'},
           …

Set initial hardening parameters to:
- $\theta_0 = 400$
- $\tau_{\textrm{sat}} = 95$

## Add simulated tests to the `HardeningLawFitter`

In [5]:
table_paths = [
    Path('data/simulated/0/geom_load.txt'),
    Path('data/simulated/1/geom_load.txt'),

]
table_data = [read_table(i) for i in table_paths]

### Test 1

- $\theta_0 = 400$
- $\tau_{\textrm{sat}} = 95$


In [6]:
fitter.add_simulated_tensile_test(
    true_stress=table_data[0]['Mises(Cauchy)'],
    true_strain=table_data[0]['Mises(ln(V))']
)

### Test 2

- $\theta_0$ = 500
- $\tau_{\textrm{sat}}$ = 140

In [7]:
fitter.add_simulated_tensile_test(
    true_stress=table_data[1]['Mises(Cauchy)'],
    true_strain=table_data[1]['Mises(ln(V))']
)

## Plot simulated stress-strain curves on their own

In [8]:
go.FigureWidget(
    data=[
        {
            'x': i['Mises(ln(V))'],
            'y': i['Mises(Cauchy)'],
            'mode': 'lines+markers',
            'text': i['inc'],
        } for i in table_data
    ]
)

FigureWidget({
    'data': [{'mode': 'lines+markers',
              'text': array([  0.,   1.,   2., ..., 258.…